In [ ]:
import torch
from torch import nn
from lib.nursing import *
from lib.utils import *
from lib.models import FCN

In [ ]:
# a single fold
foldi = 0
model = ResNet()
model.load_state_dict(torch.load(f'../projects/res_cv_{foldi}/best_model.pt'))
criterion = nn.BCEWithLogitsLoss()
test_idx = load_cv_test_idx(foldi=foldi)
X,y = load_and_window_nursing_list_for_convolution(test_idx)
devloader = DataLoader(TensorDataset(X,y),batch_size=32,shuffle=False)
loss,metric,y_true,y_pred,y_logits = evaluate(devloader,model,criterion)

In [ ]:
# over all folds
all_metrics = pd.DataFrame()

for foldi in range(5):
    model = FCN()
    model.load_state_dict(torch.load(f'../projects/fcn_cv_{foldi}/best_model.pt',map_location='cuda'))
    criterion = nn.BCEWithLogitsLoss()
    test_idx = load_cv_test_idx(foldi=foldi)
    X,y = load_and_window_nursing_list_for_convolution(test_idx)
    devloader = DataLoader(TensorDataset(X,y),batch_size=32,shuffle=False)
    loss,metric,y_true,y_pred,y_logits = evaluate(devloader,model,criterion)

    all_metrics = pd.concat([all_metrics,pd.Series(metric,name=f'{foldi}')],axis=1)

In [ ]:
all_metrics

In [ ]:
all_metrics.T.describe()

In [ ]:
import seaborn as sns
plt.figure(figsize=(5,5))
sns.boxplot(data=all_metrics.T.melt(),x='value',y='variable')

In [ ]:
a = all_metrics.reset_index().melt(id_vars='index')
a['variable'] = a['variable'].astype(int)
plt.figure(figsize=(16,4))
plt.grid()
sns.barplot(data=a,x='variable',y='value',hue='index')
plt.xlabel('fold index')
plt.ylabel('score')

In [ ]:
# let's checkout fold 2
foldi = 2
model = ResNet()
model.load_state_dict(torch.load(f'../projects/res_cv_{foldi}/best_model.pt'))
criterion = nn.BCEWithLogitsLoss()
test_idx = load_cv_test_idx(foldi=foldi)

all_metrics = pd.DataFrame()

for i in test_idx:
    X,y = load_and_window_nursing_list_for_convolution([i])
    testloader = DataLoader(TensorDataset(X,y),batch_size=32,shuffle=False)
    loss,metric,y_true,y_pred,y_logits = evaluate(testloader,model,criterion)
    fig,axes = plt.subplots(nrows=2,ncols=1,figsize=(30,5),gridspec_kw={'height_ratios': [1,5]})
    axes[0].plot(y_true,'r',linewidth=.5)
    axes[0].axis('off')
    plt.stackplot(torch.arange(len(y_logits)),torch.hstack([torch.ones_like(y_logits) - y_logits,y_logits]).T)
    # plt.savefig(f'{i}.svg',bbox_inches='tight')
    cm_grid(y_true,y_pred,save_path=f'{i}.cm.jpg')
    all_metrics = pd.concat([all_metrics,pd.Series(metric,name=f'{i}')],axis=1)

In [ ]:
all_metrics

In [ ]:
all_metrics.T.describe()

In [ ]:
plt.figure(figsize=(5,5))
sns.boxplot(data=all_metrics.T.melt(),x='value',y='variable')

In [ ]:
a = all_metrics.reset_index().melt(id_vars='index')
a['variable'] = a['variable'].astype(int)
plt.figure(figsize=(16,4))
plt.grid()
sns.barplot(data=a,x='variable',y='value',hue='index')
plt.xlabel('fold index')
plt.ylabel('score')

In [ ]:
# let's check participant 21 in fold 2
i = 22
foldi = 2
model = ResNet()
model.load_state_dict(torch.load(f'../projects/res_cv_{foldi}/best_model.pt'))
criterion = nn.BCEWithLogitsLoss()
X,y = load_and_window_nursing_list_for_convolution([i])
testloader = DataLoader(TensorDataset(X,y),batch_size=32,shuffle=False)
loss,metric,y_true,y_pred,y_logits = evaluate(testloader,model,criterion)
fig,axes = plt.subplots(nrows=2,ncols=1,figsize=(30,5),gridspec_kw={'height_ratios': [1,5]})
axes[0].plot(y_true,'r',linewidth=.5)
axes[0].axis('off')
plt.stackplot(torch.arange(len(y_logits)),torch.hstack([torch.ones_like(y_logits) - y_logits,y_logits]).T)
plt.savefig(f'logits.pdf',bbox_inches='tight')
cm_grid(y_true,y_pred,save_path=f'{i}.cm.svg')
import plotly.express as px
X,y = load_nursing_by_index(i)
df = pd.DataFrame(X.numpy())
df['y_pred'] = torch.cat([torch.zeros(101//2),y_pred.flatten().round(),torch.zeros(101//2)])*10
df['y_true'] = y*10
fig = px.line(df)
fig.show(renderer='browser')

In [ ]:
import plotly.express as px
X,y = load_nursing_by_index(0)
df = pd.DataFrame(X.numpy())
df['y_true'] = y*10
fig = px.line(df)
fig.show(renderer='browser')